In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 인코더 추가
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
#상수항추가
from statsmodels.tools.tools import add_constant
# 카이제곱, ANOVA
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
#Turkeyhsd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
# 1. 모든 parquet 파일 경로 가져오기
file_list = glob.glob('open/train/1.회원정보/2018*_train_회원정보.parquet')

# 2. 데이터프레임 리스트 생성
df_list = [pd.read_parquet(f) for f in file_list]

# 3. 하나로 합치기
all_df = pd.concat(df_list, ignore_index=True)

# 4. 확인
print(all_df.shape)
all_df

(2400000, 78)


,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),22
1,201807,TRAIN_000001,1,30대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),18
2,201807,TRAIN_000002,1,30대,C,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,20
3,201807,TRAIN_000003,2,40대,D,1,1,0,1,2,...,0,0,0,0개,0개,0개,0개,1,자녀성장(2),17
4,201807,TRAIN_000004,2,40대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,1,자녀성장(1),15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,70대이상,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,노년생활,39
2399996,201812,TRAIN_399996,2,50대,D,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),24
2399997,201812,TRAIN_399997,1,30대,C,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,18
2399998,201812,TRAIN_399998,1,40대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),27


In [3]:

def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(k, r) - 1)))

def eta_squared(anova_ss_between, total_ss):
    return anova_ss_between / total_ss if total_ss != 0 else np.nan

In [4]:
# 수치형/범주형 자동 구분
num_cols = all_df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = all_df.select_dtypes(exclude=[np.number]).columns.tolist()

results = []

In [14]:
# 데이터프레임의 컬럼 이름을 리스트에 담는다. 
column_list = all_df.columns.tolist()

In [16]:
for col in column_list:
    try:
        var_value = all_df[col].var()
        print(f"{col}: {var_value:.2f}")
    except TypeError:
        print(f"❌ 분산 계산 불가 (문자열 포함): {col}")

기준년월: 2.92
❌ 분산 계산 불가 (문자열 포함): ID
남녀구분코드: 0.25
❌ 분산 계산 불가 (문자열 포함): 연령
❌ 분산 계산 불가 (문자열 포함): Segment
회원여부_이용가능: 0.05
회원여부_이용가능_CA: 0.11
회원여부_이용가능_카드론: 0.24
소지여부_신용: 0.02
소지카드수_유효_신용: 0.32
소지카드수_이용가능_신용: 0.38
입회일자_신용: 3655901162.18
입회경과개월수_신용: 5243.27
회원여부_연체: 0.02
이용거절여부_카드론: 0.16
동의여부_한도증액안내: 0.09
수신거부여부_TM: 0.23
수신거부여부_DM: 0.22
수신거부여부_메일: 0.22
수신거부여부_SMS: 0.23
❌ 분산 계산 불가 (문자열 포함): 가입통신회사코드
탈회횟수_누적: 0.45
최종탈회후경과월: 1467.08
탈회횟수_발급6개월이내: 0.04
탈회횟수_발급1년이내: 0.06
❌ 분산 계산 불가 (문자열 포함): 거주시도명
❌ 분산 계산 불가 (문자열 포함): 직장시도명
마케팅동의여부: 0.15
유효카드수_신용체크: 1.53
유효카드수_신용: 0.77
유효카드수_신용_가족: 0.05
유효카드수_체크: 0.64
유효카드수_체크_가족: 0.00
이용가능카드수_신용체크: 1.57
이용가능카드수_신용: 0.85
이용가능카드수_신용_가족: 0.05
이용가능카드수_체크: 0.64
이용가능카드수_체크_가족: 0.00
이용카드수_신용체크: 1.00
이용카드수_신용: 0.81
이용카드수_신용_가족: 0.05
이용카드수_체크: 0.14
이용카드수_체크_가족: 0.00
이용금액_R3M_신용체크: 540450252.29
이용금액_R3M_신용: 479537906.53
이용금액_R3M_신용_가족: 3114786.48
이용금액_R3M_체크: 56674031.67
이용금액_R3M_체크_가족: 0.00
_1순위카드이용금액: 198528629.01
_1순위카드이용건수: 2601.09
❌ 분산 계산 불가 (문자열 포함): _1순위신용체크구분
_2순위카

In [18]:
def print_value_counts(df, columns=None, dropna=False):

    if columns is None:
        columns = df.columns.tolist()
    
    for col in columns:
        print(f"\n[🔢 Value Counts for '{col}']\n")
        print(df[col].value_counts(dropna=dropna))

In [20]:
zero_variance_cols  = []
# 각 컬럼별 분산 계산
for col in column_list:
    try:
        var_value = all_df[col].var()
        if var_value == 0:
            zero_variance_cols.append(col)
    except TypeError:
        # 숫자형이 아닌 경우 무시
        pass

# 결과 출력
print("📌 분산이 0인 컬럼:")
for col in zero_variance_cols:
    print(f"{col}")

📌 분산이 0인 컬럼:
이용카드수_체크_가족
이용금액_R3M_체크_가족
연회비할인카드수_B0M
할인금액_기본연회비_B0M
할인금액_제휴연회비_B0M


In [22]:
print_value_counts(all_df, columns=zero_variance_cols)


[🔢 Value Counts for '이용카드수_체크_가족']

이용카드수_체크_가족
0    2400000
Name: count, dtype: int64

[🔢 Value Counts for '이용금액_R3M_체크_가족']

이용금액_R3M_체크_가족
0    2400000
Name: count, dtype: int64

[🔢 Value Counts for '연회비할인카드수_B0M']

연회비할인카드수_B0M
0    2400000
Name: count, dtype: int64

[🔢 Value Counts for '할인금액_기본연회비_B0M']

할인금액_기본연회비_B0M
0    2400000
Name: count, dtype: int64

[🔢 Value Counts for '할인금액_제휴연회비_B0M']

할인금액_제휴연회비_B0M
0    2400000
Name: count, dtype: int64


- 빼야할 컬럼 
['이용카드수_체크_가족',
'이용금액_R3M_체크_가족',
'연회비할인카드수_B0M',
'할인금액_기본연회비_B0M',
'할인금액_제휴연회비_B0M']

In [32]:
cols_to_drop = ['이용카드수_체크_가족',
'이용금액_R3M_체크_가족',
'연회비할인카드수_B0M',
'할인금액_기본연회비_B0M',
'할인금액_제휴연회비_B0M']

all_df = all_df.drop(columns=cols_to_drop)

In [26]:
print(cat_cols)

['ID', '연령', 'Segment', '가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분', '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', 'Life_Stage']


In [28]:
print(num_cols)

['기준년월', '남녀구분코드', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월']


In [33]:
# 리스트에서 제거
num_cols = [col for col in num_cols if col not in cols_to_drop]

In [36]:
print(num_cols)

['기준년월', '남녀구분코드', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '_1순위카드이용금액', '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '기본연회비_B0M', '제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월']


In [38]:
anova = []
chi = []

for col in all_df.columns:
    if col == 'Segment':
        continue

    if col in num_cols:
        data = all_df[[col, 'Segment']].dropna()
        groups = [data[data['Segment'] == val][col] for val in data['Segment'].unique()]
        try:
            stat = f_oneway(*groups).statistic
            ss_between = sum([(g.mean() - data[col].mean())**2 * len(g) for g in groups])
            ss_total = sum((data[col] - data[col].mean())**2)
            eta2 = eta_squared(ss_between, ss_total)
            anova.append({'변수': col, '유형': '수치형', '계수종류': 'Eta²', '상관계수': eta2})
        except:
            continue

    elif col in cat_cols:
        contingency = pd.crosstab(all_df[col], all_df['Segment'])
        if contingency.shape[0] > 1 and contingency.shape[1] > 1:
            try:
                v = cramers_v(contingency)
                chi.append({'변수': col, '유형': '범주형', '계수종류': "Cramér's V", '상관계수': v})
            except:
                continue

# 결과 정리
result_df1 = pd.DataFrame(anova)
result_df2 = pd.DataFrame(chi)
result_df1 = result_df1.sort_values(by='상관계수', ascending=False).reset_index(drop=True)
result_df2 = result_df2.sort_values(by='상관계수', ascending=False).reset_index(drop=True)

# 결과 출력
display(result_df1)
display(result_df2)

,변수,유형,계수종류,상관계수
0,이용금액_R3M_신용체크,수치형,Eta²,0.389813
1,이용금액_R3M_신용,수치형,Eta²,0.347924
2,_1순위카드이용금액,수치형,Eta²,0.329734
3,이용카드수_신용체크,수치형,Eta²,0.166022
4,_2순위카드이용금액,수치형,Eta²,0.165938
5,_1순위카드이용건수,수치형,Eta²,0.154889
6,_2순위카드이용건수,수치형,Eta²,0.151003
7,이용카드수_신용,수치형,Eta²,0.147888
8,이용가능카드수_신용체크,수치형,Eta²,0.129013
9,이용가능카드수_신용,수치형,Eta²,0.127935


,변수,유형,계수종류,상관계수
0,ID,범주형,Cramér's V,1.000000
1,_2순위신용체크구분,범주형,Cramér's V,0.154185
2,가입통신회사코드,범주형,Cramér's V,0.053978
3,Life_Stage,범주형,Cramér's V,0.049693
4,_1순위신용체크구분,범주형,Cramér's V,0.046067
5,직장시도명,범주형,Cramér's V,0.045916
6,연령,범주형,Cramér's V,0.045830
7,거주시도명,범주형,Cramér's V,0.034416
8,연회비발생카드수_B0M,범주형,Cramér's V,0.008985


### 범주형 컬럼 지정한 경우 

In [43]:
# 첫 번째 리스트 (삭제 대상 후보)
target_list = [
    '기준년월', '남녀구분코드', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론',
    '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용',
    '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM',
    '수신거부여부_메일', '수신거부여부_SMS', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내',
    '탈회횟수_발급1년이내', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족',
    '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용',
    '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크',
    '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용금액_R3M_신용체크',
    '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '_1순위카드이용금액',
    '_1순위카드이용건수', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능',
    '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인',
    '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인',
    '이용여부_3M_해외겸용_신용_본인', '기본연회비_B0M', '제휴연회비_B0M',
    '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', '최종카드발급경과월'
]

In [45]:
# 두 번째 리스트 (삭제 기준 목록)
drop_criteria = [
    '성별', '동의여부', '남녀구분코드', '연령', 'Life_Stage', '가입통신회사코드',
    '거주시도명', '직장시도명', '회원여부_이용가능', '회원여부_이용가능_CA',
    '회원여부_이용가능_카드론', '회원여부_연체', '탈퇴횟수_누적', '탈퇴회수_발급6개월이내',
    '탈회횟수_발급1년이내', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용',
    '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_보인',
    '연회비발생카드수_BOM', '_1순위신용체크구분', '_2순위신용체크구분',
    '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM',
    '수신거부여부_메일', '수신거부여부_SMS', '마케팅동의여부', 'ID'
]

# 정확히 일치하는 항목만 제거
filtered_list = [col for col in target_list if col not in drop_criteria]

In [51]:
# 수치형 중 범주형으로 처리할 컬럼 지정
manual_cat_cols = [
    '성별', '동의여부', '남녀구분코드', '연령', 'Life_Stage',
    '가입통신회사코드', '거주시도명', '직장시도명',
    '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론',
    '회원여부_연체', '탈퇴횟수_누적', '탈회회수_발급6개월이내', '탈회횟수_발급1년이내',
    '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용',
    '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_보인',
    '연회비발생카드수_BOM', '_1순위신용체크구분', '_2순위신용체크구분',
    '이용거절여부_카드론', '동의여부_한도증액안내',
    '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS',
    '마케팅동의여부', 'ID'
]

anova = []
chi = []

# 컬럼 반복
for col in all_df.columns:
    if col == 'Segment':
        continue

    if col in filtered_list:
        data = all_df[[col, 'Segment']].dropna()
        groups = [data[data['Segment'] == val][col] for val in data['Segment'].unique()]
        try:
            stat = f_oneway(*groups).statistic
            ss_between = sum([(g.mean() - data[col].mean())**2 * len(g) for g in groups])
            ss_total = sum((data[col] - data[col].mean())**2)
            eta2 = ss_between / ss_total
            anova.append({'변수': col, '유형': '수치형', '계수종류': 'Eta²', '상관계수': eta2})
        except:
            continue

    elif col in manual_cat_cols:
        contingency = pd.crosstab(all_df[col], all_df['Segment'])
        if contingency.shape[0] > 1 and contingency.shape[1] > 1:
            try:
                v = cramers_v(contingency)
                chi.append({'변수': col, '유형': '범주형', '계수종류': "Cramér's V", '상관계수': v})
            except:
                continue

# 결과 데이터프레임
result_df3 = pd.DataFrame(anova)
result_df4 = pd.DataFrame(chi)

# 정렬
result_df3 = result_df3.sort_values(by='상관계수', ascending=False).reset_index(drop=True)
result_df4 = result_df4.sort_values(by='상관계수', ascending=False).reset_index(drop=True)

# 출력
display(result_df3)
display(result_df4)

,변수,유형,계수종류,상관계수
0,이용금액_R3M_신용체크,수치형,Eta²,0.389813
1,이용금액_R3M_신용,수치형,Eta²,0.347924
2,_1순위카드이용금액,수치형,Eta²,0.329734
3,이용카드수_신용체크,수치형,Eta²,0.166022
4,_2순위카드이용금액,수치형,Eta²,0.165938
5,_1순위카드이용건수,수치형,Eta²,0.154889
6,_2순위카드이용건수,수치형,Eta²,0.151003
7,이용카드수_신용,수치형,Eta²,0.147888
8,이용가능카드수_신용체크,수치형,Eta²,0.129013
9,이용가능카드수_신용,수치형,Eta²,0.127935


,변수,유형,계수종류,상관계수
0,ID,범주형,Cramér's V,1.000000
1,소지카드수_이용가능_신용,범주형,Cramér's V,0.200278
2,소지카드수_유효_신용,범주형,Cramér's V,0.173724
3,이용가능여부_해외겸용_본인,범주형,Cramér's V,0.164520
4,_2순위신용체크구분,범주형,Cramér's V,0.154185
5,보유여부_해외겸용_본인,범주형,Cramér's V,0.147875
6,수신거부여부_TM,범주형,Cramér's V,0.110458
7,수신거부여부_메일,범주형,Cramér's V,0.110142
8,수신거부여부_DM,범주형,Cramér's V,0.108484
9,수신거부여부_SMS,범주형,Cramér's V,0.096999


In [53]:
# 엑셀로 저장
result_df1.to_excel("open/1.회원정보_Anova_결과.xlsx", index=False)
result_df2.to_excel("open/1.회원정보_카이제곱_결과.xlsx", index=False)
result_df3.to_excel("open/1.회원정보_Anova_결과_컬럼지정.xlsx", index=False)
result_df4.to_excel("open/1.회원정보_카이제곱_결과_컬럼지정.xlsx", index=False)

In [55]:
# Label Encoding
le = LabelEncoder()

segment_order = {'E': 0, 'D': 1, 'C': 2, 'B': 3, 'A': 4}
all_df['Segment_encoded'] = all_df['Segment'].map(segment_order)

# 수치형 변수 + Segment_encoded만 선택
num_cols_with_seg = num_cols + ['Segment_encoded']

# 상관계수 계산
corr_df = all_df[num_cols_with_seg].corr()

# Segment_encoded 기준 상관계수 추출
segment_corr = corr_df[['Segment_encoded']].drop(index='Segment_encoded').reset_index()
segment_corr.columns = ['변수', '상관계수']

# 정렬: 절대값 기준으로만 정렬 (값은 원래 그대로)
segment_corr = segment_corr.reindex(segment_corr['상관계수'].abs().sort_values(ascending=False).index).reset_index(drop=True)

display(segment_corr.style.format({'상관계수': "{:.3f}"}))


,변수,상관계수
0,이용금액_R3M_신용체크,0.623
1,이용금액_R3M_신용,0.589
2,_1순위카드이용금액,0.574
3,_2순위카드이용금액,0.400
4,이용카드수_신용체크,0.398
5,_1순위카드이용건수,0.384
6,이용카드수_신용,0.377
7,_2순위카드이용건수,0.376
8,이용가능카드수_신용,0.353
9,이용가능카드수_신용체크,0.352


### Anova vs Pearson 비교 
- 일단, 수치형 컬럼만 비교 진행했음.

In [ ]:
# ===== Pearson 준비 =====
pearson_df = segment_corr[['변수', '상관계수']].copy()
pearson_df.columns = ['변수', 'Pearson']
pearson_df['Pearson_rank'] = pearson_df['Pearson'].abs().rank(ascending=False, method='min')
pearson_df['Pearson_rank'] = pearson_df['Pearson_rank'].fillna(0).astype(int)

# ===== ANOVA 준비 =====
anova_df = result_df1[['변수', '상관계수']].copy()
anova_df.columns = ['변수', 'Eta2']
anova_df['Eta2_rank'] = anova_df['Eta2'].rank(ascending=False, method='min')
anova_df['Eta2_rank'] = anova_df['Eta2_rank'].fillna(0).astype(int)

# ===== 병합 =====
merged_df = pd.merge(anova_df, pearson_df, on='변수', how='outer')

# ===== 순위 차이 =====
merged_df['Rank_diff'] = (merged_df['Eta2_rank'] - merged_df['Pearson_rank']).abs()

# ===== NaN 제거 & 정렬 =====
merged_df_clean = merged_df.dropna(subset=['Eta2', 'Pearson'])
merged_df_clean = merged_df_clean.sort_values(by='Eta2', ascending=False, na_position='last').reset_index(drop=True)

# ===== 출력 =====
display(merged_df_clean.style.format({'Eta2': "{:.3f}", 'Pearson': "{:.3f}"}))
